In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [2]:
rating_full_df = pd.read_csv(r'D:\WorkLab\RecSystem\data\ratings.csv')
movie_df = pd.read_csv(r'D:\WorkLab\RecSystem\data\movies.csv')

In [3]:
rating_df = rating_full_df[:1000000]
rating_df.drop('timestamp', axis=1, inplace =True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_820\343773286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_df.drop('timestamp', axis=1, inplace =True)


In [4]:
unique_genres = set()
movie_df['genres'] = movie_df['genres'].apply(lambda x: x.split('|'))
movie_df['genres'].apply(unique_genres.update)



0        None
1        None
2        None
3        None
4        None
         ... 
62418    None
62419    None
62420    None
62421    None
62422    None
Name: genres, Length: 62423, dtype: object

In [5]:
unique_genres = sorted(unique_genres)
genre_to_id = {genre: i for i, genre in enumerate(unique_genres)}
movie_genres_ids = [[genre_to_id[genre] for genre in genres] for genres in movie_df['genres']]

In [6]:
X = pad_sequences(movie_genres_ids, padding='post')


In [7]:
X

array([[ 2,  3,  4, ...,  0,  0,  0],
       [ 2,  4,  9, ...,  0,  0,  0],
       [ 5, 15,  0, ...,  0,  0,  0],
       ...,
       [ 5,  8,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  2,  8, ...,  0,  0,  0]], dtype=int32)

In [38]:
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(input_dim=len(unique_genres)+1, output_dim=8)(input_layer)
flatten = Flatten()(embedding_layer)

In [39]:
embedding_model = Model(inputs=input_layer, outputs=flatten)

In [40]:
embeddings = embedding_model.predict(X)
movie_df['embedding'] = embeddings.tolist()

1951/1951 ━━━━━━━━━━━━━━━━━━━━ 2s 919us/step


In [45]:
movie_df

,movieId,title,genres,embedding
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[-0.0466427206993103, 0.025392401963472366, 0...."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[-0.0466427206993103, 0.025392401963472366, 0...."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[-0.0197464469820261, -0.04607229307293892, -0..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[-0.0197464469820261, -0.04607229307293892, -0..."
4,5,Father of the Bride Part II (1995),[Comedy],"[-0.0197464469820261, -0.04607229307293892, -0..."
...,...,...,...,...
62418,209157,We (2018),[Drama],"[0.0036846771836280823, -0.0424957275390625, 0..."
62419,209159,Window of the Soul (2001),[Documentary],"[0.04912317916750908, -0.0037118904292583466, ..."
62420,209163,Bad Poems (2018),"[Comedy, Drama]","[-0.0197464469820261, -0.04607229307293892, -0..."
62421,209169,A Girl Thing (2001),[(no genres listed)],"[0.007933378219604492, 0.00995192676782608, 0...."


In [46]:
embedding_model.save('embedd_model.keras')

In [47]:
movie_df.to_csv('movie_df_with_embeddings', index=False)